In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#{"R": [R], "D": [D],"H": [H], "F":[log10(f)],"RP": [RP]})
DATA = pd.read_csv("DATA/data.csv")
DATA.pop("Unnamed: 0")

ROW_NUM = len(DATA.index)
print(ROW_NUM)
TRAINING_NUM = int(ROW_NUM / 10 * 7)
VALIDATION_NUM = int(ROW_NUM / 20 * 3 )

TRAINING_DATA = DATA.iloc[ :TRAINING_NUM,:]
VALIDATION_DATA = DATA.iloc[TRAINING_NUM:TRAINING_NUM+VALIDATION_NUM,:]
TEST_DATA = DATA.iloc[TRAINING_NUM+VALIDATION_NUM:,:]
TRAINING_VAL = pd.DataFrame(TRAINING_DATA.pop('RP'))
VALIDATION_VAL = pd.DataFrame(VALIDATION_DATA.pop('RP'))
TEST_VAL = pd.DataFrame(TEST_DATA.pop('RP'))

DATA.pop('RP')


In [6]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
min_max_scaler.fit(DATA)
joblib.dump(min_max_scaler, "MLP.scaler.gz") 

TRAINING_DATA = min_max_scaler.transform(TRAINING_DATA)
TRAINING_DATA = pd.DataFrame(TRAINING_DATA)
VALIDATION_DATA = min_max_scaler.transform(VALIDATION_DATA)
VALIDATION_DATA = pd.DataFrame(VALIDATION_DATA)
TEST_DATA = min_max_scaler.transform(TEST_DATA)
TEST_DATA = pd.DataFrame(TEST_DATA)

In [ ]:

model = keras.models.Sequential([
    keras.layers.Dense(320,activation="tanh", input_dim=4),
    keras.layers.Dense(1, activation="linear")
])

learning_rate = 0.0001
optimizer = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss="mean_absolute_error", optimizer=optimizer)

epochs=100
batch=32


model_name = f"mae_{model.layers[0].input_shape[1]}"
for layer in model.layers:
    model_name+=f"-{layer.output_shape[1]}"
model_name += f"_e:{epochs}_b:{batch}_lr:{learning_rate}"
print(model_name)


history = model.fit(TRAINING_DATA, TRAINING_VAL, epochs=epochs, batch_size=batch, validation_data=(VALIDATION_DATA, VALIDATION_VAL))
model.save(f"MODELS/{model_name}.h5")

from math import log10

scores = model.evaluate(TEST_DATA, TEST_VAL, batch_size=20)
plt.plot([10*log10(loss) for loss in history.history['loss'][4:]])
plt.plot([10*log10(loss) for loss in history.history['val_loss'][4:]])
plt.title(f'model loss {scores}')
plt.ylabel('loss dB')
plt.xlabel('epoch')
plt.legend(['TRAIN', 'VALID'], loc='upper left')
plt.savefig(f"TRAINING_PROCESS/{model_name}.png")
plt.show()


In [16]:
TEST_DATA = TEST_DATA.iloc[:30,:]
PREDICTIONS = pd.DataFrame(model.predict(TEST_DATA, verbose=0))
PREDICTIONS.columns=["PREDICITONS"]

TEST_VAL = TEST_VAL.iloc[:30,:]
TEST_VAL.columns=["TEST_VAL"]
TEST_VAL = TEST_VAL.reset_index(drop=True)

result=pd.concat([PREDICTIONS, TEST_VAL], axis=1)
result["ERROR"] = result["PREDICITONS"] - result["TEST_VAL"]
result.to_csv("MLP.csv")
